In [1]:
pip install imutils

In [2]:
pip install opencv-python

In [3]:
import tensorflow as tf

In [4]:
pip install pillow

In [5]:
from PIL import Image

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [7]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [8]:
import imutils

In [9]:
from imutils import paths


In [10]:

import os
import numpy as np

In [11]:
!pip install matplotlib

In [12]:
import matplotlib.pyplot as plt

In [13]:
init_lr=1e-4
bs=32
epochs=20

In [14]:
directory='datasets/mask_detection_video'
categories=['with_mask','without_mask']


In [15]:
data=[]
labels=[]
for category in categories:
    path=os.path.join(directory,category)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path,target_size=(224,224))
        
        
        image=img_to_array(image)
        image=preprocess_input(image)
        data.append(image)
        labels.append(category)

C:\Users\rasaa\OneDrive\Desktop\prepinsta project\OpenCV project\facemask\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [16]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [17]:
data=np.array(data,dtype='float32')
labels=np.array(labels)

In [18]:
(trainX,testX,trainY,testY)=train_test_split(data,labels,test_size=0.2,stratify=labels,random_state=42)

In [19]:
aug=ImageDataGenerator(rotation_range=20,
                      zoom_range=0.15,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.15,
                      horizontal_flip=True,
                      fill_mode='nearest')

In [20]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,
                     input_tensor=Input(shape=(224,224,3)))

In [21]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)

In [22]:
headModel=Flatten(name='flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

In [23]:
model=Model(inputs=baseModel.input,outputs=headModel)

In [24]:
for layer in baseModel.layers:
    layer.trainable=False

In [25]:
opt=Adam(learning_rate=init_lr,decay=init_lr/epochs)
model.compile(optimizer=opt,metrics=['accuracy'],loss='binary_crossentropy')

In [26]:
H=model.fit(aug.flow(trainX,trainY,batch_size=bs),
            steps_per_epoch=len(trainX)//bs,
            validation_data=(testX,testY),
            validation_steps=len(testX)//bs,
            epochs=epochs
           )

Epoch 1/20
95/95 [==============================] - 104s 1s/step - loss: 0.4112 - accuracy: 0.8556 - val_loss: 0.1535 - val_accuracy: 0.9896
Epoch 2/20
95/95 [==============================] - 104s 1s/step - loss: 0.1545 - accuracy: 0.9657 - val_loss: 0.0813 - val_accuracy: 0.9883
Epoch 3/20
95/95 [==============================] - 97s 1s/step - loss: 0.1103 - accuracy: 0.9710 - val_loss: 0.0595 - val_accuracy: 0.9909
Epoch 4/20
95/95 [==============================] - 98s 1s/step - loss: 0.0732 - accuracy: 0.9832 - val_loss: 0.0488 - val_accuracy: 0.9909
Epoch 5/20
95/95 [==============================] - 91s 962ms/step - loss: 0.0755 - accuracy: 0.9743 - val_loss: 0.0415 - val_accuracy: 0.9909
Epoch 6/20
95/95 [==============================] - 87s 915ms/step - loss: 0.0626 - accuracy: 0.9822 - val_loss: 0.0390 - val_accuracy: 0.9922
Epoch 7/20
95/95 [==============================] - 86s 907ms/step - loss: 0.0534 - accuracy: 0.9848 - val_loss: 0.0343 - val_accuracy: 0.9909
Epoch 8/2

In [27]:
pred=model.predict(testX,batch_size=bs)
pred_classes = np.argmax(pred, axis=1)

24/24 [==============================] - 16s 603ms/step


In [28]:
print(classification_report(testY.argmax(axis=1),pred_classes,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [29]:
model.save('mask_detector_new.model',save_format='h5')